In [1]:
!pip install autogen langchain llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [2]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 7.9 MB/s eta 0:00:00


In [3]:
!pip install accelerate

In [4]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.0/175.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/

In [5]:
import yfinance as yf
from transformers import AutoModelForCausalLM, AutoTokenizer,  TextStreamer
from autogen import Agent
import requests
import torch
from unsloth import FastLanguageModel

import pprint

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# Load the Meta LLaMA 3.1 8B model and tokenizer
model_name = "unsloth/Llama-3.2-1B-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-3B-Instruct"
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [16]:
model = FastLanguageModel.for_inference(model)

In [17]:
# Define Data Analyst Agent
class DataAnalystAgent(Agent):
    def __init__(self, model, tokenizer, device):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.device = device

    def analyze_data(self, client_info):
        # Analyze client information and generate insights
        prompt = f"Analyze the following client information and provide a financial advice report:\n{client_info}"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(**inputs, max_new_tokens=500)
        insights = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return insights

In [18]:
# Define Recommendation Agent
class RecommendationAgent(Agent):
    def __init__(self, model, tokenizer, data_analyst, device):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.data_analyst = data_analyst
        self.device = device

    def generate_recommendation(self, client_info):
        insights = self.data_analyst.analyze_data(client_info)
        prompt = f"Generate a recommendation for the following client: {client_info} based on the following insights: {insights}"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(**inputs, max_new_tokens=500)
        recommendation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return recommendation

In [19]:
# Define Reporting Agent
class ReportingAgent:
    def __init__(self):
        pass

    def generate_report(self, client_info, insights, recommendations):
        report = f"Client Financial Report\n"
        report += f"========================\n"
        report += f"Client Information:\n{client_info}\n\n"
        report += f"Insights:\n{insights}\n\n"
        report += f"Recommendations:\n{recommendations}\n"
        report += f"========================\n"
        report += f"End of Report"
        return report

In [21]:
# client information
client_info1 = """
Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Engineer
- Email: john.chan@abc.com
- Annual Income: HKD $780,000
- Monthly Expenses: HKD $45,000
- Monthly Savings: HKD $15,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $100,000 (60% in stocks, 40% in bonds)
  - Stock Portfolio: HKD $600,000 (Tech stocks: $400,000, Diversified ETFs: $200,000)
- Debt:
  - Mortgage: HKD $2,000,000 (5% interest)
  - Car Loan: HKD $100,000 (3% interest)
- Financial Goals:
  - Short-Term: Build an emergency fund of HKD $700,000 within 1 year.
  - Medium-Term: Save HKD $1,000,000 for a down payment on a house in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $10,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%

"""

In [22]:
client_info2 = """
Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HKD $480,000
- Monthly Expenses: HKD $30,000
- Monthly Savings: HKD $20,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $50,000 (70% in stocks, 30% in bonds)
  - Stock Portfolio: HKD $150,000 (Tech stocks: $100,000, Diversified ETFs: $50,000)
- Debt:
  - Student Loan: HKD $200,000 (4% interest)
- Financial Goals:
  - Short-Term: Save HKD $100,000 for travel within 1 year.
  - Medium-Term: Save HKD $500,000 for a home down payment in 3 years.
  - Long-Term: Build a retirement fund of HKD $5,000,000 by age 60.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%

"""

In [23]:
client_info3 = """
Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,200,000
- Monthly Expenses: HKD $70,000
- Monthly Savings: HKD $30,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $300,000 (50% in stocks, 50% in bonds)
  - Stock Portfolio: HKD $1,200,000 (Blue-chip stocks: $800,000, Diversified ETFs: $400,000)
- Debt:
  - Mortgage: HKD $1,500,000 (3.5% interest)
- Financial Goals:
  - Short-Term: Pay off mortgage within 5 years.
  - Medium-Term: Save HKD $2,000,000 for retirement in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $20,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%
"""

In [25]:
# Define workflow
def workflow(client_info, model, tokenizer, device):
    # Initialize agents
    data_analyst = DataAnalystAgent(model, tokenizer, device)
    recommendation_agent = RecommendationAgent(model, tokenizer, data_analyst, device)
    reporting_agent = ReportingAgent()

    # Step 1: Analyze client data
    insights = data_analyst.analyze_data(client_info)

    # Step 2: Generate recommendations based on insights
    recommendations = recommendation_agent.generate_recommendation(client_info)

    # Step 3: Generate a report
    report = reporting_agent.generate_report(client_info, insights, recommendations)

    return {
        "insights": insights,
        "recommendations": recommendations,
        "report": report,
    }

In [26]:
# Run workflow for client 1
result1 = workflow(client_info1, model, tokenizer, device)
# print(result1)

{'insights': "Analyze the following client information and provide a financial advice report:\n\nClient Information:\n- Name: John Chan\n- Age: 35\n- Occupation: Software Engineer\n- Email: john.chan@abc.com\n- Annual Income: HKD $780,000\n- Monthly Expenses: HKD $45,000\n- Monthly Savings: HKD $15,000\n- Investments:\n  - Mandatory Provident Fund (MPF): HKD $100,000 (60% in stocks, 40% in bonds)\n  - Stock Portfolio: HKD $600,000 (Tech stocks: $400,000, Diversified ETFs: $200,000)\n- Debt:\n  - Mortgage: HKD $2,000,000 (5% interest)\n  - Car Loan: HKD $100,000 (3% interest)\n- Financial Goals:\n  - Short-Term: Build an emergency fund of HKD $700,000 within 1 year.\n  - Medium-Term: Save HKD $1,000,000 for a down payment on a house in 5 years.\n  - Long-Term: Retire at age 60 with a monthly income of HKD $10,000.\n- Assumptions:\n- Inflation Rate: 2.5%\n- Interest Rate: 3.0%\n- Tax Rate: 15%\n\nFinancial Advice Report:\n\n**Overview**\n\nBased on the provided client information, John C

In [31]:
formatted_report1 = result1['report'].replace('\\n', '\n')
print(formatted_report1)

Client Financial Report
Client Information:

Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Engineer
- Email: john.chan@abc.com
- Annual Income: HKD $780,000
- Monthly Expenses: HKD $45,000
- Monthly Savings: HKD $15,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $100,000 (60% in stocks, 40% in bonds)
  - Stock Portfolio: HKD $600,000 (Tech stocks: $400,000, Diversified ETFs: $200,000)
- Debt:
  - Mortgage: HKD $2,000,000 (5% interest)
  - Car Loan: HKD $100,000 (3% interest)
- Financial Goals:
  - Short-Term: Build an emergency fund of HKD $700,000 within 1 year.
  - Medium-Term: Save HKD $1,000,000 for a down payment on a house in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $10,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%



Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Eng

In [36]:
# Run workflow for client 2
result2 = workflow(client_info2, model, tokenizer, device)

In [37]:
formatted_report2 = result2['report'].replace('\\n', '\n')
print(formatted_report2)

Client Financial Report
Client Information:

Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HKD $480,000
- Monthly Expenses: HKD $30,000
- Monthly Savings: HKD $20,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $50,000 (70% in stocks, 30% in bonds)
  - Stock Portfolio: HKD $150,000 (Tech stocks: $100,000, Diversified ETFs: $50,000)
- Debt:
  - Student Loan: HKD $200,000 (4% interest)
- Financial Goals:
  - Short-Term: Save HKD $100,000 for travel within 1 year.
  - Medium-Term: Save HKD $500,000 for a home down payment in 3 years.
  - Long-Term: Build a retirement fund of HKD $5,000,000 by age 60.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%



Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HK

In [38]:
# Run workflow for client 3
result3 = workflow(client_info3, model, tokenizer, device)

In [39]:
formatted_report3 = result3['report'].replace('\\n', '\n')
print(formatted_report3)

Client Financial Report
Client Information:

Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,200,000
- Monthly Expenses: HKD $70,000
- Monthly Savings: HKD $30,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $300,000 (50% in stocks, 50% in bonds)
  - Stock Portfolio: HKD $1,200,000 (Blue-chip stocks: $800,000, Diversified ETFs: $400,000)
- Debt:
  - Mortgage: HKD $1,500,000 (3.5% interest)
- Financial Goals:
  - Short-Term: Pay off mortgage within 5 years.
  - Medium-Term: Save HKD $2,000,000 for retirement in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $20,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%


Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,2

In [ ]:
# ------------------------------------------------------------